In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


## Importing the data from BIG CSV

In [9]:
data = pd.read_csv(
    'Raw data/openipf-2024-12-28-acdecc3a.csv',
    dtype={'BodyweightKg': 'float64', 'TotalKg': 'float64'},
    low_memory=False
)

In [20]:
# Parse the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')


In [21]:
# Filter for raw, full-power meets (SBD), males, and international meets from 2011 onwards
filtered_data = data[
    (data['Event'] == 'SBD') &              # Full power (SBD)
    (data['Equipment'] == 'Raw') &         # Raw lifting
    (data['Sex'] == 'M') &
    (data['BodyweightKg'] >= 37.5)&
    # (data['Federation'].isin(['IPF', 'EPF'])) &  # International meets only
    (data['Date'] >= '2011-01-01') &
    (data['Date'] <= '2024-06-15')
    # (data['Date'] <= '2020-04-30')
]

In [22]:
# Ensure necessary columns are present and drop missing values
filtered_data = filtered_data[['Name', 'BodyweightKg', 'TotalKg', 'WeightClassKg', 'Federation', 'Date','MeetName']].dropna()

filtered_data['MeetName'] = filtered_data['MeetName'].str.strip().str.lower()

In [23]:
filtered_data.loc[:, 'Name'] = (
    filtered_data['Name']
    .str.strip()  # Remove leading/trailing spaces
    .str.lower()  # Convert to lowercase for uniformity
)

In [61]:

# Retain the highest total per person per weight class
filtered_data = filtered_data.loc[
    filtered_data.groupby(['Name', 'WeightClassKg'])['TotalKg'].idxmax()
].reset_index(drop=True)


In [62]:
# Save the filtered data to a CSV
filtered_data.to_csv('all_meets_beforeworldsfemale_ipf_epf.csv', index=False)
print("Filtered data saved as 'filtered_data_ipf_epf.csv'.")


Filtered data saved as 'filtered_data_ipf_epf.csv'.


In [24]:
filtered_data.sort_values('BodyweightKg', ascending=False, inplace=True)
top10 = filtered_data.head(10)
display(top10)

,Name,BodyweightKg,TotalKg,WeightClassKg,Federation,Date,MeetName
486664,javier alberto salazar pasten,260.0,695.0,120+,FEMEPO,2022-03-18,campeonato nacional
376369,joshua sprouse,233.0,392.5,120+,USAPL,2016-02-20,alabama state championships
376367,joshua sprouse,233.0,392.5,120+,USAPL,2016-02-20,alabama state championships
366187,robert stolley,231.0,450.0,120+,USAPL,2018-04-28,nebraska state championship
386282,joshua sprouse,230.7,357.5,120+,USAPL,2015-05-09,border bash
1156626,timothy edwards #1,226.7,565.0,120+,AMP,2022-06-18,nationals
295044,joshua sprouse,225.0,385.0,125+,USAPL,2014-03-15,alabama state powerlifting championship
304095,jorge munoz,220.4,535.0,120+,USAPL,2020-08-08,return to the platform
359869,joshua sprouse,220.2,347.5,120+,USAPL,2017-06-03,gulf breeze open
182235,rajmund schwarz,217.5,610.0,120+,Hunpower,2023-04-29,hungarian national classic powerlifting champi...


In [124]:


# World records male 2024
world_records = {
     '59': 669.5, '66': 710.5, '74': 836.0, '83': 861.0, '93': 901.0,
    '105': 940.5, '120': 978.5, '120+': 1152.5  # Example values
}

In [111]:
# World records male 2020
world_records = {
    '59': 669.5, '66': 705.5, '74': 790.5, '83': 833, '93': 853.5,
    '105': 895.5, '120': 978.5, '120+': 1105.5
}

In [72]:

# World records female 2020
world_records = {'47': 407.5, '52': 432.5, '57': 473.5,
                 '63': 543.5, '72': 543.5, '84': 613, '84+': 671.5
}

In [63]:
# world records female 2024
world_records = {'47': 433.5, '52': 481, '57': 519.5,
                 '63': 557.5, '69': 600, '76': 613, '84': 647, '84+': 731
}

In [64]:
# filtering for gold standard data
filtered_data['WeightClassKg'] = filtered_data['WeightClassKg'].astype(str)
gold_standard_data = filtered_data[
    filtered_data.apply(
        lambda row: row['TotalKg'] >= 0.84 * world_records.get(row['WeightClassKg'], np.inf), axis=1
    )
]

In [65]:
# Standardize 'Name' column to ensure proper deduplication
gold_standard_data.loc[:, 'Name'] = (
    gold_standard_data['Name']
    .str.strip()  # Remove leading/trailing spaces
    .str.lower()  # Convert to lowercase for uniformity
)

In [66]:
# Deduplicate to retain only the top total for each lifter
# Deduplicate to retain only the top total for each lifter per weight class
gold_standard_data = gold_standard_data.loc[
    gold_standard_data.groupby(['Name', 'WeightClassKg'])['TotalKg'].idxmax()
].reset_index(drop=True)


In [67]:
# Save the gold-standard data to a CSV
gold_standard_data.to_csv('gold_standard_before_worlds_female_data_ipf_epf.csv', index=False)
print("Gold standard data saved as 'gold_standard_data_ipf_epf.csv'.")



Gold standard data saved as 'gold_standard_data_ipf_epf.csv'.
